In [1]:
%cd /content/drive/MyDrive/
# raw_data is imported from global config
root_dir='/content/drive/MyDrive/'  # ChangeThis
result_dir=root_dir+'grasp/braindecode/result_deep/'
import os
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

/content/drive/MyDrive


In [2]:
! pip install mne==0.19.2;
! pip install torch;

     |████████████████████████████████| 6.4MB 10.3MB/s 


In [3]:
import os
os.getcwd()

'/content/drive/MyDrive'

In [4]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

from grasp.utils import raw_input,freq_input, SEEGDataset3D, cuda_or_cup, set_random_seeds
from grasp.TSception.utils import regulization
from grasp.TSception.Models import TSception
from grasp.braindecode.Models import shallowConv,deepConv
# load the data
from grasp.utils import rawData

Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json


In [5]:
import sys, importlib
importlib.reload(sys.modules['grasp.TSception.Models'])
#from grasp.TSception.Models import TSception2


<module 'grasp.TSception.Models' from '/content/drive/MyDrive/grasp/TSception/Models.py'>

In [6]:
import inspect as i
import sys
#sys.stdout.write(i.getsource(rawData2));

In [7]:
enable_cuda = torch.cuda.is_available()
print('GPU computing: ', enable_cuda)

GPU computing:  True


In [8]:
%%capture
# suppress the output
import sys, importlib
importlib.reload(sys.modules['grasp.utils'])
from grasp.utils import freq_input
sampling_rate=1000

#traindata, valdata, testdata = freq_input(6,split=True,move2=True)
traindata, valdata, testdata = raw_input(6,split=True,move2=True)
traindata = traindata.transpose(2, 0, 1)  # (118, 20, 15000) (trials,channels,  time)
valdata = valdata.transpose(2, 0, 1) # (8, 20, 15000)
testdata = testdata.transpose(2, 0, 1)  # (8, 20, 15000)

trainx, trainy = traindata[:, :-2, :], traindata[:, -2, :] #-2 is real force, -1 is target
valx, valy = valdata[:, :-2, :], valdata[:, -2, :]
testx, testy = testdata[:, :-2, :], testdata[:, -2, :]

step=200 #ms
T=500 #ms

dataset_train = SEEGDataset3D(trainx, trainy,T,step)
dataset_val = SEEGDataset3D(valx, valy,T,step)
dataset_test = SEEGDataset3D(testx, testy,T,step)


In [9]:
# Dataloader for training process
train_loader = DataLoader(dataset=dataset_train, batch_size=1, shuffle=True, pin_memory=False)
val_loader = DataLoader(dataset=dataset_val, batch_size=1, pin_memory=False)
test_loader = DataLoader(dataset=dataset_test, batch_size=1, pin_memory=False)

chnNum=trainx.shape[1]
learning_rate=0.0001
epochs=100
totalLen=trainx.shape[2] #ms
batch_size=int((totalLen-T)/step) # 28
convfeature=40
kernelSize=200
avgpoolKernel=100
dropout=0.5
Lambda = 1e-6

checkshape=torch.squeeze(next(iter(test_loader))[0])
length=checkshape.shape[2] # torch.Size([28, 90, 1000])


net=shallowConv(length, chnNum, convfeature, kernelSize, avgpoolKernel,dropout).cuda()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
criterion = nn.MSELoss()


In [10]:

debugg=False
#debugg=True
for epoch in range(epochs):
    print("------ epoch " + str(epoch) + " -----")
    net.train()

    loss_epoch=0
    #trial=0
    for trial, (trainx, target) in enumerate(train_loader): # ([1, 15000, 19]), ([1, 15000])
        if debugg==True: # just test one trial
            if trial == 1:
                break
                pass
        optimizer.zero_grad()
        #print("Training on trial " + str(trial) + ".")
        y_pred = net(trainx.cuda())
        # regularization
        loss1 = criterion(torch.squeeze(y_pred), torch.squeeze(target.cuda()))
        loss2 = regulization(net, Lambda)
        #loss3 = y_pred.cpu().detach().numpy()
        #loss3 = np.std(np.diff(loss3.reshape(-1)))
        loss=loss1+loss2 #+loss3*0.001
        loss.backward()
        optimizer.step()

        ls=loss1.item()
        loss_epoch+=ls
        with open(result_dir+"trainlose.txt", "a") as f:
            f.write(str(loss1) + "\n")
    print("----- Epoch "+str(epoch)+" loss:"+str(loss_epoch/(trial+1))+". -----")
    if epoch % 1 ==0:
        loss_val=0
        net.eval()
        with torch.no_grad():
            vpredAll = []
            vtargetAll = []
            for trialv, (valx, vtarget) in enumerate(val_loader):  # ([1, 15000, 19]), ([1, 15000])
                #print("Validating on trial " + str(trial) + ".")

                vpred = net(valx.cuda())
                vpredAll.append(vpred.cpu().data)
                vtargetAll.append(torch.squeeze(vtarget))
                loss3 = criterion(torch.squeeze(vpred), torch.squeeze(vtarget.cuda()))
                loss_val+=loss3.item()
                with open(result_dir+"testlose.txt", "a") as f:
                    f.write(str(loss3) + "\n")
        vpredAll = np.concatenate(vpredAll,axis=0)
        vtargetAll = np.concatenate(vtargetAll, axis=0)

        fig, ax = plt.subplots(figsize=(6, 3))
        plt.ion()
        ax.clear()
        ax.plot(vtargetAll, label="True", linewidth=1)
        ax.plot(vpredAll, label='Predicted - Test', linewidth=1)
        ax.legend(loc='upper left')
        figname = result_dir+'prediction' + str(epoch) + '.pdf'
        fig.savefig(figname)
        plt.close(fig)
    print("----- Epoch "+str(epoch)+"Train loss:"+str(loss_epoch/(trial+1))+", Validate loss: "+ str(loss_val/(trialv+1))+"-----")
    if epoch % 5==0:
        state = {
            'state_dict': net.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        savepath = result_dir+'checkpoint'+str(epoch)+'.pth'
        torch.save(state, savepath)


------ epoch 0 -----
----- Epoch 0 loss:0.6262746571989383. -----
----- Epoch 0Train loss:0.6262746571989383, Validate loss: 0.2858979208394885-----
------ epoch 1 -----
----- Epoch 1 loss:0.4719277293247692. -----
----- Epoch 1Train loss:0.4719277293247692, Validate loss: 0.2796568041667342-----
------ epoch 2 -----
----- Epoch 2 loss:0.3968371728719291. -----
----- Epoch 2Train loss:0.3968371728719291, Validate loss: 0.23053561244159937-----
------ epoch 3 -----
----- Epoch 3 loss:0.36120809185302866. -----
----- Epoch 3Train loss:0.36120809185302866, Validate loss: 0.2808419596403837-----
------ epoch 4 -----
----- Epoch 4 loss:0.3437838268355798. -----
----- Epoch 4Train loss:0.3437838268355798, Validate loss: 0.23941212985664606-----
------ epoch 5 -----
----- Epoch 5 loss:0.30160645111385037. -----
----- Epoch 5Train loss:0.30160645111385037, Validate loss: 0.2103922013193369-----
------ epoch 6 -----
----- Epoch 6 loss:0.283494703029677. -----
----- Epoch 6Train loss:0.283494703